In [2]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import time
from django.contrib.gis.geos import Point, fromstr, GEOSGeometry
from datetime import datetime, timedelta
import vaex
import vaex.ml
from joblib import delayed, Parallel, load, parallel_backend
# from haversine import haversine, Unit

dir = Path.cwd()
outdir = os.path.join(dir, 'output')
gtfs_records_zip = os.path.join(dir, 'data', 'GtfsRRecords.zip')
gtfs_csv_zip = os.path.join(outdir, 'gtfsr_csv.zip')
gtfs_final_csv_path = os.path.join(outdir, 'gtfsr.csv')
gtfs_processed_csv_path = os.path.join(outdir, "gtfsr_processed.csv")
scats = os.path.join(dir, 'output', 'scats_model.json')

In [3]:
df = vaex.from_csv(gtfs_processed_csv_path, convert=True)
df = df.sample(frac=1)
df

#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,stop_id,arrival_time,departure_time,shape_dist_traveled,lat,lon,direction_angle,shape_dist_between,hour,dow,p_avg_vol
0,14181.3.60-79-b12-1.331.I,20210131,15:20:00,27,0,0,2021-01-31 15:38:23,8220DB007453,15:43:53,15:43:53,9236.25,53.3463358924194,-6.27838250402172,86.7035700423138,425.4500000000007,15,6,96.25931192252952
1,8746.2.60-16-b12-1.129.I,20210130,10:48:00,52,-240,-240,2021-01-30 11:24:56,8220DB000021,11:38:01,11:38:01,14109.77,53.3701011992972,-6.25431059904437,2.2863971656259885,327.3000000000011,11,5,39.43149146620169
2,8003.10455.2-17-gad-1.3.I,20210129,12:20:00,31,300,300,2021-01-29 12:56:38,8250DB001035,12:54:00,12:54:00,11129.83,53.2951037415434,-6.25612557753287,-74.30741720193481,428.09000000000015,12,4,91.5310097916111
3,20600.4.60-15-b12-1.32.O,20210122,16:45:00,35,-60,-60,2021-01-22 17:19:30,8220DB007581,17:22:09,17:22:09,11537.96,53.3441585614977,-6.26334590761396,-126.28881488258796,483.4599999999991,17,4,86.68459828990551
4,17261.4.60-44-b12-1.246.O,20210128,08:30:00,38,420,420,2021-01-28 09:14:29,8220DB002815,09:12:32,09:12:32,12194.89,53.31585100315,-6.24526859253124,155.4154238326518,486.1299999999992,9,3,103.8975076370368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"1,025,849",20546.4.60-15-b12-1.31.I,20210122,16:30:00,61,480,480,2021-01-22 17:37:04,8220DB001202,17:30:28,17:30:28,18606.24,53.3925013826657,-6.1934290504328,53.530212757221655,260.4800000000032,17,4,98.48893553284805
"1,025,850",18482.4.60-4-b12-1.5.O,20210125,16:30:00,29,300,300,2021-01-25 16:57:28,8220DB004725,16:55:36,16:55:36,10308.95,53.352115616113,-6.26105453403103,135.97323997287558,696.5600000000013,16,0,43.25699451487691
"1,025,851",19796.4.60-46A-b12-1.258.O,20210128,16:10:00,27,120,120,2021-01-28 16:32:27,8220DB000759,16:42:30,16:42:30,8172.56,53.3203488508448,-6.23334551683728,109.19283828441368,396.4500000000007,16,3,44.58032089316716
"1,025,852",16708.4.60-31-b12-1.159.O,20210122,11:20:00,26,-60,-60,2021-01-22 11:44:30,8220DB000537,11:44:46,11:44:46,7683.29,53.3799012003082,-6.16815263497718,83.68808488620553,361.75,11,4,69.81196590113757


In [70]:
# len(df)
df.trip_id.value_counts()

17254.4.60-44-b12-1.246.O       409
18284.4.60-16-b12-1.128.O       399
21076.4.60-33-b12-1.171.I       399
19980.4.60-32X-b12-1.36.I       382
17159.4.60-40-b12-1.209.I       370
                               ... 
14889.3.60-68-b12-1.313.I         1
6496.10451.2-451-gad-1.276.I      1
7641.10466.2-270-gad-1.235.I      1
6529.10454.2-451-gad-1.276.I      1
6203.10451.2-451-gad-1.278.O      1
Length: 18978, dtype: int64

In [5]:
df.describe().T

,dtype,count,NA,mean,std,min,max
trip_id,str,1025854,0,--,--,--,--
start_date,int64,1025854,0,20210126.800474532,41.630067,20210122,20210131
start_time,str,1025854,0,--,--,--,--
stop_sequence,int64,1025854,0,31.50653211860557,20.63737,1,104
departure,int64,1025854,0,145.26119701244036,241.596064,-9000,5700
arrival,int64,1025854,0,139.45832447892195,233.956756,-7200,5400
timestamp,str,1025854,0,--,--,--,--
stop_id,str,1025854,0,--,--,--,--
arrival_time,str,1025854,0,--,--,--,--
departure_time,str,1025854,0,--,--,--,--


In [6]:
# df[df.stop_sequence == 1]

In [7]:
# df[(df['hour'] == 0) & (df['dow'] == 0)]

In [72]:
one_trip = df[df['trip_id'] == '17254.4.60-44-b12-1.246.O']
one_trip.export_hdf5('./output/one_trip.hdf5')
one_trip


#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,stop_id,arrival_time,departure_time,shape_dist_traveled,lat,lon,direction_angle,shape_dist_between,hour,dow,p_avg_vol
0,17254.4.60-44-b12-1.246.O,20210126,14:30:00,51,180,180,2021-01-26 15:20:22,8250DB002829,15:23:35,15:23:35,16370.46,53.2843852487127,-6.2368693911913,155.4154238326518,422.5599999999995,15,1,134.26521523870264
1,17254.4.60-44-b12-1.246.O,20210127,14:30:00,68,-180,-180,2021-01-27 15:46:05,8250DB003478,15:38:31,15:38:31,22858.06,53.2390505484196,-6.19616830395543,155.4154238326518,487.3900000000031,15,2,46.032404798293385
2,17254.4.60-44-b12-1.246.O,20210128,14:30:00,21,0,0,2021-01-28 14:34:05,8220DB000265,14:48:14,14:48:14,6140.71,53.3533615685169,-6.26205538070456,155.4154238326518,568.5799999999999,14,3,69.36955095945075
3,17254.4.60-44-b12-1.246.O,20210129,14:30:00,74,-360,-360,2021-01-29 15:38:59,8250DB004088,15:41:55,15:41:55,25207.9,53.2203949175122,-6.18166193356728,155.4154238326518,333.7300000000032,15,4,48.74178690612492
4,17254.4.60-44-b12-1.246.O,20210122,14:30:00,63,240,240,2021-01-22 15:32:57,8250DB003473,15:34:20,15:34:20,20655.89,53.254120160543,-6.21441883615046,155.4154238326518,671.1499999999978,15,4,73.92295609760234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,17254.4.60-44-b12-1.246.O,20210126,14:30:00,68,-480,-480,2021-01-26 15:41:09,8250DB003478,15:38:31,15:38:31,22858.06,53.2390505484196,-6.19616830395543,155.4154238326518,487.3900000000031,15,1,43.76902421209255
405,17254.4.60-44-b12-1.246.O,20210128,14:30:00,29,540,540,2021-01-28 15:04:01,8220DB001013,15:02:40,15:02:40,9039.17,53.335481505514,-6.25755688072012,155.4154238326518,191.1800000000003,15,3,79.11236079328911
406,17254.4.60-44-b12-1.246.O,20210128,14:30:00,20,180,180,2021-01-28 14:44:18,8220DB000052,14:45:11,14:45:11,5572.13,53.3566607792373,-6.26456528204094,155.4154238326518,156.76000000000022,14,3,55.71405186191136
407,17254.4.60-44-b12-1.246.O,20210125,14:30:00,14,0,0,2021-01-25 14:36:42,8220DB000045,14:39:47,14:39:47,3922.37,53.3699364344031,-6.25410694084961,155.4154238326518,193.94000000000003,14,0,33.103805778136696


In [73]:
df_train, df_test = vaex.open('./output/one_trip.hdf5').ml.train_test_split(test_size=0.2, verbose=False)

In [74]:
from sklearn.linear_model import SGDRegressor
from vaex.ml.sklearn import IncrementalPredictor

model_out = os.path.join(outdir, "gtfsr_model.json")
start = time.time()

def train_gtfsr(df):

    # transform the features into more machine learning friendly vars
    pca_coord = vaex.ml.PCA(features=["lat", "lon"], n_components=2, prefix="pca")
    df = pca_coord.fit_transform(df)


    cycl_transform_angle = vaex.ml.CycleTransformer(features=["direction_angle"], n=360)
    df = cycl_transform_angle.fit_transform(df)

    df['t_dow'] = df['timestamp'].apply(lambda t: datetime.strptime(t, "%Y-%m-%d %H:%M:%S").weekday())
    df['t_hour'] = df['timestamp'].apply(lambda t: datetime.strptime(t, "%Y-%m-%d %H:%M:%S").hour)
    df['t_minute'] = df['timestamp'].apply(lambda t: datetime.strptime(t, "%Y-%m-%d %H:%M:%S").minute)
    df['t_second'] = df['timestamp'].apply(lambda t: datetime.strptime(t, "%Y-%m-%d %H:%M:%S").second)

    df['arr_dow'] = df['start_date'].apply(lambda t: datetime.strptime(str(t), "%Y%m%d").weekday())
    df['arr_hour'] = df['arrival_time'].apply(lambda t: datetime.strptime(t, "%H:%M:%S").hour)
    df['arr_minute'] = df['arrival_time'].apply(lambda t: datetime.strptime(t, "%H:%M:%S").minute)
    df['arr_second'] = df['arrival_time'].apply(lambda t: datetime.strptime(t, "%H:%M:%S").second)

    cycl_transform_dow = vaex.ml.CycleTransformer(features=["t_dow", 'arr_dow'], n=7)
    df = cycl_transform_dow.fit_transform(df)

    cycl_transform_hour = vaex.ml.CycleTransformer(features=["t_hour", 'arr_hour'], n=24)
    df = cycl_transform_hour.fit_transform(df)

    cycl_transform_minute = vaex.ml.CycleTransformer(features=["t_minute", 't_second', 'arr_minute', 'arr_second'], n=60)
    df = cycl_transform_minute.fit_transform(df)

    # one_hot_encode = vaex.ml.OneHotEncoder(features=['trip_id'])
    # df = one_hot_encode.fit_transform(df)

    standard_scaler = vaex.ml.StandardScaler(features=['shape_dist_traveled', 'shape_dist_between', "p_avg_vol"])
    df = standard_scaler.fit_transform(df)

    feats = df.get_column_names(regex="pca") + \
        df.get_column_names(regex=".*_x") + \
        df.get_column_names(regex=".*_y") + \
        df.get_column_names(regex="standard_scaled_*") + \
        ["stop_sequence"]
        # df.get_column_names(regex="trip_id_*") + \
        
    target = "arrival"

    print("dataWrangling done, ready to create model, time: {}s".format(round(time.time() - start)))

    # create a randomForestRegression model
    model = SGDRegressor(learning_rate='constant', eta0=0.0001)
    vaex_model = IncrementalPredictor(
        features=feats, 
        target=target,
        model=model, 
        prediction_name='p_arrival',
        batch_size=11_000_000, 
        num_epochs=1000,
    )

    # here we fit and train the model
    with parallel_backend("threading", n_jobs=8):
        vaex_model.fit(df=df, progress='widget')
        print("\n\nmodel created, time: {}s".format(round(time.time() - start)))

        # dump(value=vaex_model, filename=model_out, compress=3)
        # print("model written to output, time: {}s".format(round(time.time() - start)))

        trained = vaex_model.transform(df)
    return trained[[
            'arrival', 
            'p_arrival', 
            'trip_id',
            'start_date',
            'start_time',
            'stop_sequence',
            'timestamp',
            'stop_id',
            'arrival_time',
            'departure_time',
            'p_avg_vol' ]]

train_gtfsr(df_train)


dataWrangling done, ready to create model, time: 0s




model created, time: 41s


#,arrival,p_arrival,trip_id,start_date,start_time,stop_sequence,timestamp,stop_id,arrival_time,departure_time,p_avg_vol
0,-120,-230.75786095247312,17254.4.60-44-b12-1.246.O,20210126,14:30:00,69,2021-01-26 15:37:03,8250DB003280,15:38:46,15:38:46,43.76902421209255
1,60,-134.90010969592765,17254.4.60-44-b12-1.246.O,20210128,14:30:00,66,2021-01-28 15:36:16,8250DB003476,15:36:27,15:36:27,71.81210925216853
2,120,191.56137833878384,17254.4.60-44-b12-1.246.O,20210126,14:30:00,21,2021-01-26 14:34:59,8220DB000265,14:48:14,14:48:14,71.87005014879405
3,180,125.33502906807601,17254.4.60-44-b12-1.246.O,20210122,14:30:00,55,2021-01-22 15:20:36,8250DB002833,15:26:27,15:26:27,106.95620082164946
4,300,269.2917981285822,17254.4.60-44-b12-1.246.O,20210122,14:30:00,50,2021-01-22 15:25:44,8250DB007717,15:21:31,15:21:31,158.34839235095944
...,...,...,...,...,...,...,...,...,...,...,...
322,-480,-344.40459652744687,17254.4.60-44-b12-1.246.O,20210126,14:30:00,68,2021-01-26 15:41:09,8250DB003478,15:38:31,15:38:31,43.76902421209255
323,540,573.113766589181,17254.4.60-44-b12-1.246.O,20210128,14:30:00,29,2021-01-28 15:04:01,8220DB001013,15:02:40,15:02:40,79.11236079328911
324,180,134.4987000860859,17254.4.60-44-b12-1.246.O,20210128,14:30:00,20,2021-01-28 14:44:18,8220DB000052,14:45:11,14:45:11,55.71405186191136
325,0,-48.37578796256773,17254.4.60-44-b12-1.246.O,20210125,14:30:00,14,2021-01-25 14:36:42,8220DB000045,14:39:47,14:39:47,33.103805778136696


In [24]:
df.plot_widget(df.lon, df.lat, shape=512, colormap='plasma', f='log1p', limits='minmax')

Heatmap(children=[ToolsToolbar(interact_value=None, supports_normalize=False, template='<template>\n  <v-toolb…